<a href="https://colab.research.google.com/github/Adilmar/TensorFlow1/blob/master/create_model_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# extract data from a csv
# notice the cool options to skip lines at the beginning
# and to only take data from certain columns
training = np.genfromtxt('/content/drive/My Drive/Datasets/data_pt.csv', delimiter=',', skip_header=1, usecols=(1, 3), dtype=None, encoding='utf-8')

# create our training data from the tweets
train_x = [x[1] for x in training]
# index all the sentiment labels
train_y = np.asarray([x[0] for x in training])

print(train_y)

#only work with the 3000 most popular words found in our dataset
max_words = 3000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('/content/drive/My Drive/Datasets/outputs/tw/dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = keras.utils.to_categorical(train_y, 2)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.fit(train_x, train_y,
    batch_size=32,
    epochs=1000,
    verbose=1,
    validation_split=0.1,
    shuffle=True)

model_json = model.to_json()
with open('/content/drive/My Drive/Datasets/outputs/tw/model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('/content/drive/My Drive/Datasets/outputs/tw/model.h5')

print('saved model!')


[0 0 1 ... 1 1 1]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 18478 samples, validate on 2054 samples
Epoch 1/1000
18478/18478 [==============================] - 4s 216us/step - loss: 0.6119 - acc: 0.6679 - val_loss: 0.6322 - val_acc: 0.6490
Epoch 2/1000
18478/18478 [==============================] - 3s 148us/step - loss: 0.4847 - acc: 0.7599 - val_loss: 0.6229 - val_acc: 0.6602
Epoch 3/1000
18478/18478 [==============================] - 3s 147us/step - loss: 0.4245 - acc: 0.7949 - val_loss: 0.6842 - val_acc: 0.6470
Epoch 4/1000
18478/18478 [==============================] - 3s 149us/step - loss: 0.3504 - acc: 0.8377 - val_loss: 0.7196 - val_acc: 0.6538
Epoch 5/1000
18478/18478 [==============================] - 3s 150us/step - loss: 0.2754 - acc: 0.8736 - val_loss: 0.8499 - val_acc: 0.643